In [1]:
import numpy as np
import fasttext
import pandas as pd
import random
from resources.tokTT import CommentTokenizer as CT
from resources.basicIO import InputOutput as IO
from resources.basicIO import InputOutput as IO
from resources.filterLang import FilterLanguage as FL
from active_expansion.fasttext_batch_avg import Expander
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.base import BaseEstimator, TransformerMixin
import fasttext
import pandas as pd
from scipy import spatial
from scipy.spatial import distance
from sklearn import svm
from sklearn import metrics
from sklearn.linear_model import SGDClassifier
#from sklearn.linear_model import LassoLars
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import cross_val_score
import copy
import scipy


[nltk_data] Downloading package wordnet to C:\Users\AJAY
[nltk_data]     BISWAS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
def score(model, line, k):
    # words contains all the words in the corpus
    lst1 = model.get_nearest_neighbors(line, k)
    v1 = []
    l1 = [x[1] for x in lst1]
    l10 = [x[0] for x in lst1]
    for i in range(len(model.words)):
        try:
            v1.append(l10[l1.index(model.words[i])])
        except:
            v1.append(0)
    return v1


def cos_sim(model, l1, l2, k):
    a = score(model, l1, k)
    b = score(model, l2, k)
    
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)

    if(norm_a * norm_b == 0.0):
        return dot_product / (norm_a * norm_b + 0.001)

    return dot_product / (norm_a * norm_b)


def sim(model, x, y, k, sim_type):
    if(sim_type == 'cosine_sim'):
        return cos_sim(model, x, y, k)


def sim_matrix(model, A, B, k, sim_type):
    m = len(A)
    n = len(B)
    C = np.zeros((m, n))
    for i in range(m):
        for j in range(n):
            C[i][j] = sim(model, A[i], B[j], k, sim_type)
    return C


In [39]:
def Expand(model, seed_set_TK, seed_set_label, expansion_TK, expansion_text_labels, batch_size, count, k, random_rate):
    seed_TK = copy.deepcopy(seed_set_TK)
    seed_labels = copy.deepcopy(seed_set_label)
    M = np.arange(0, count, batch_size)
    cnt = int(random_rate * batch_size)
    count2 = [0]
    print(M)

    for i in range(1, len(M)):

        exp_TK = expansion_TK[M[i-1]:M[i]]
        exp_labels = expansion_text_labels[M[i-1]:M[i]]

        C = sim_matrix(model, seed_TK, exp_TK, k, 'cosine_sim')
        Y_ind = np.argmax(C, axis=0)
        Y = [seed_labels[x] for x in Y_ind]

        seed_labels.extend(Y)
        seed_TK.extend(exp_TK)

    return seed_TK, seed_labels, count2


In [40]:
model_N_2 = fasttext.load_model('models/ft_unsupervised_N_2.bin')


In [33]:
A = ['I support farmers', 'I am proud of farmers', 'i hate farmers', 'i want farm bills']
A_y = [0,0,1,1]
B = ['I am support farm bill', 'I love farmers', 'they are not farmers', 'farm bills are good']
B_y = [1,0,1,1]


In [41]:
X_train_exp, y_train_exp, count2 = Expand(model_N_2,
                                  A,
                                  A_y,
                                  B,
                                  B_y,
                                  2,
                                  len(B_y),
                                  20,
                                  0.1
                                  )


[0 2]


In [43]:
print(y_train_exp)

[0, 0, 1, 1, 1, 0]
